Алгоритм позволяет оценить $\theta$, где $\theta = \Theta/(2 \pi)$ (пусть в формулировке задания используется $\Theta$) входит в показатель экспоненты собственного числа оператора $U|\psi\rangle=e^{2 \pi i \theta}|\psi\rangle$. В случае $T$-оператора из данной задачи $|\psi\rangle=|1\rangle$.


Шаги алгоритма:

К начальному состоянию в форме $|0\rangle^{\otimes n}|1\rangle$ мы применяем операторы Адамара ко всем кубитам кроме последнего

$$
f_{1}=\frac{1}{\sqrt{2^n}}(|0\rangle+|1\rangle)^{\otimes n}|\psi\rangle
$$
Применяя оператор $U$ несколько раз, получим -- 
$U^{2^{j}}|\psi\rangle=e^{2 \pi i 2^{j} \theta}|\psi\rangle$.
Применяем контролируемые операции $C-U^{2^{j}}$ к $f_{1}$:
$$
\frac{1}{2^{\frac{n}{2}}}\left(|0\rangle+e^{2 \pi i \theta 2^{n-1}}|1\rangle\right) \otimes \cdots \otimes\left(|0\rangle+e^{2 \pi i \theta 2^{1}}|1\rangle\right) \otimes\left(|0\rangle+e^{2 \pi i \theta 2^{0}}|1\rangle\right) \otimes|\psi\rangle 
 \equiv \frac{1}{2^{\frac{n}{2}}} \sum_{k=0}^{2^{n}-1} e^{2 \pi i \theta k}|k\rangle \otimes|\psi\rangle
$$

Заметим, что то выражение совпадает с применением КПФ к $x$ для $x=2^{n} \theta$, то есть для получения состояния $\left|2^{n} \theta\right\rangle,$ нужно сделать обратное преобразование Фурье.


$$
QFT^{-1}\frac{1}{2^{\frac{n}{2}}} \sum_{k=0}^{2^{n}-1} e^{2 \pi i \theta k}|k\rangle \otimes|\psi\rangle = \frac{1}{2^n} \sum_{x=0}^{2^{n}-1} \sum_{k=0}^{2^{n}-1} e^{-\frac{2 \pi i k}{2^{n}}\left(x-2^{n} \theta\right)}|x\rangle \otimes|\psi\rangle
$$
При измерении максимальная вероятность будет для значений $x \approx 2^{n} \theta$.




---------------------

Рассмотрим подробно в качестве примера случай:
$$
T|1\rangle=\left[\begin{array}{cc}
1 & 0 \\
0 & e^{\frac{i \pi}{4}}
\end{array}\right]\left[\begin{array}{l}
0 \\
1
\end{array}\right]=e^{\frac{i \pi}{4}}|1\rangle
$$


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import *
from qiskit import *
from qiskit.visualization import *
Theta=pi/4

In [2]:
qcir = QuantumCircuit(4, 3)
qcir.x(3)

qcir.h(0)
qcir.h(1)
qcir.h(2)    

qcir.cu1(Theta, 0, 3)
qcir.cu1(Theta, 1, 3)
qcir.cu1(Theta, 1, 3)
qcir.cu1(Theta, 2, 3)
qcir.cu1(Theta, 2, 3)
qcir.cu1(Theta, 2, 3)
qcir.cu1(Theta, 2, 3)
qcir.draw()

┌───┐                                                 
q_0: ┤ H ├─■───────────────────────────────────────────────
     ├───┤ │                                               
q_1: ┤ H ├─┼──────■──────■─────────────────────────────────
     ├───┤ │      │      │                                 
q_2: ┤ H ├─┼──────┼──────┼──────■──────■──────■──────■─────
     ├───┤ │pi/4  │pi/4  │pi/4  │pi/4  │pi/4  │pi/4  │pi/4 
q_3: ┤ X ├─■──────■──────■──────■──────■──────■──────■─────
     └───┘                                                 
c: 3/══════════════════════════════════════════════════════

In [3]:
def qft_inverse(qcirc):
    for j in range(3):
        for m in range(j):
            qcirc.cu1(-pi/float(2**(j-m)), m, j)
        qcirc.h(j)

In [4]:
qcir.swap(0,2)
qft_inverse(qcir)
qcir.measure(0,0)
qcir.measure(1,1)
qcir.measure(2,2)
qcir.draw()

┌───┐                                                    ┌───┐        »
q_0: ┤ H ├─■────────────────────────────────────────────────X─┤ H ├─■──────»
     ├───┤ │                                                │ └───┘ │-pi/2 »
q_1: ┤ H ├─┼──────■──────■──────────────────────────────────┼───────■──────»
     ├───┤ │      │      │                                  │              »
q_2: ┤ H ├─┼──────┼──────┼──────■──────■──────■──────■──────X──────────────»
     ├───┤ │pi/4  │pi/4  │pi/4  │pi/4  │pi/4  │pi/4  │pi/4                 »
q_3: ┤ X ├─■──────■──────■──────■──────■──────■──────■─────────────────────»
     └───┘                                                                 »
c: 3/══════════════════════════════════════════════════════════════════════»
                                                                           »
«                          ┌─┐           
«q_0: ──────■──────────────┤M├───────────
«     ┌───┐ │              └╥┘     ┌─┐   
«q_1: ┤ H ├─┼───────■───────╫──────┤M├───
«     └───┘ │-pi/4  │-pi/2  ║ ┌───┐└╥┘┌─┐
«q_2: ──────■───────■───────╫─┤ H ├─╫─┤M├
«                           ║ └───┘ ║ └╥┘
«q_3: ──────────────────────╫───────╫──╫─
«                           ║       ║  ║ 
«c: 3/══════════════════════╩═══════╩══╩═
«                           0       1  2

In [5]:
backend = Aer.get_backend('qasm_simulator')
shots = 1000
results = execute(qcir, backend=backend, shots=shots).result()
answer = results.get_counts()
print(answer)

{'001': 1000}


$2^{n} \theta=x$ 

Мы получили состояние 001 в качестве результата измерения ( 001 -- в бинарном виде, в десятичном это $x=1$) с веростностью 1. Чтобы получить $\theta$:
$$
\theta=\frac{1}{2^{3}}=\frac{1}{8},
$$


Для
$
T|1\rangle=e^{2 i \pi \theta}|1\rangle,
$
мы и ожидали получить для $\Theta = \dfrac{\pi}{4}$
$$
\theta=\frac{1}{8}.
$$


Аналогично мы можем взять другую фазу, получить наиболее вероятно другое число в бинарном виде, перевести его в десятичный и определить фазу.